# Import garden.db

### 3.1 all plants in db

In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('gardening.db')
c = conn.cursor()

import readline

# Connect to the database
conn = sqlite3.connect('gardening.db')
c = conn.cursor()

# Set up readline for console input
readline.parse_and_bind("tab: complete")
readline.set_history_length(1000)

# Define a function to execute a SQL query
def execute_query(query):
    try:
        print("=========================================")
        # Execute the SQL query
        c.execute(query)

        # Get column names
        column_names = [description[0] for description in c.description] if c.description else []

        # Print column names
        for col_name in column_names:
            print(col_name, end="\t")
        if column_names:
            print()  # New line after column names


        results = c.fetchall()

        # Print the results
        if len(results) == 0:
            print("No results found.")
        else:
            for row in results:
                print(row)

        print("=========================================")
        
        # Commit the changes to the database
        conn.commit()

        # Print a message for successful INSERT, UPDATE, or DELETE statements
        if c.rowcount > 0:
            print("Query executed successfully.")

        # Print the last inserted row ID for INSERT statements
        if query.lower().startswith('insert'):
            print("Last inserted row ID:", c.lastrowid)

    except sqlite3.Error as e:
        # Print an error message if the query fails
        print("An error occurred:", e)

# Loop to prompt the user for input
query_lines = []
while True:
    try:
        # Check if it's the beginning of a new command
        if not query_lines:
            prompt = "Enter a SQL query (or 'exit' to quit): \n"
        else:
            prompt = ''

        line = input(prompt)
        
        # If the user just presses Enter without typing anything, consider it as execution time
        if not line.strip() and query_lines:
            query = ' '.join(query_lines)
            query_lines.clear()
            execute_query(query)
        else:
            query_lines.append(line)

    except KeyboardInterrupt:
        # Handle Ctrl+C to exit the loop cleanly
        print("\nExiting...")
        break

    # Check if the user wants to exit
    if line.lower() == 'exit':
        break

    # Handle up arrow to load last query (this remains unchanged, but might need a different approach)
    if line == '':
        query = readline.get_history_item(readline.get_current_history_length())

    # Add the query to the history (this remains unchanged)
    readline.add_history(line)



In [2]:
query = """
SELECT pname
FROM plant
WHERE ptype IN ('עץ פרי', 'עץ נוי');
"""
execute_query(query)

pname	
('חרוב מצוי',)
('זית',)
('ברוש',)
('דולב',)
('אורן ירושלים',)


career-readiness-week-1-Dor-sketch

https://github.com/github-student-developer-pack/career-readiness-week-1-Dor-sketch




In [3]:
# Close the database connection
conn.close()


# 3 using ext sql

In [4]:
%load_ext sql

In [5]:
%sql sqlite:///gardening.db

# SUBQURRIES

## 3.22 plant with max size using subquiry

In [6]:
%%sql
SELECT pname
FROM plant
WHERE ptype IN ('עץ פרי', 'עץ נוי');

 * sqlite:///gardening.db
Done.


pname
חרוב מצוי
זית
ברוש
דולב
אורן ירושלים


In [7]:
%%sql
-- # 3.22 plant with max size using subquiry
-- ALL operator not supported!!!
select pname as tallest_plant
from plant
where pmaxheight >=all(select pmaxheight from plant);


 * sqlite:///gardening.db
(sqlite3.OperationalError) near "all": syntax error
[SQL: -- # 3.22 plant with max size using subquiry
-- ALL operator not supported!!!
select pname as tallest_plant
from plant
where pmaxheight >=all(select pmaxheight from plant);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [8]:
%%sql
-- 3.22
select pname as tallest_plant
from plant
where pmaxheight = (select max(pmaxheight) from plant);

 * sqlite:///gardening.db
Done.


tallest_plant
ברוש
דולב


## 3.23 anames where heights diffrences is the biggest

subqury inside having line

In [9]:
%%sql
select aname
from plant, plantingmap
where plant.pname = plantingmap.pname
group by aname
having (max(pmaxheight)-min(pmaxheight)) >= all
    (select )

 * sqlite:///gardening.db
(sqlite3.OperationalError) near "all": syntax error
[SQL: select aname
from plant, plantingmap
where plant.pname = plantingmap.pname
group by aname
having (max(pmaxheight)-min(pmaxheight)) >= all
    (select )]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


# EXIST - check if there is a table in a subquary
retutn true / FALSE
NEED SUBQUIRRY!!!

a * querry returns  full rows...

In [10]:
%%sql
-- 3.24 find a plant that is planted in only one anmae
SELECT p1.pname
FROM plantingmap AS p1
WHERE NOT EXISTS (
    SELECT *
    FROM plantingmap AS p2
    WHERE p1.pname = p2.pname
    AND p1.aname <> p2.aname
);

 * sqlite:///gardening.db
Done.


pname
חרוב מצוי
אחירותם
my_bush1
my_bush2
צבעוני ההרים


wrong!!!! - I groped by aname instead of pname!!!!

In [12]:
%%sql
-- 3.24 again but with grouping
--  maybe count?

select pname
from plantingmap
group by aname
having count (aname) = 1;

 * sqlite:///gardening.db
Done.


pname


In [13]:
%%sql 
-- trying again

select pname
from plantingmap
group by pname
having count(distinct aname) = 1;

 * sqlite:///gardening.db
Done.


pname
my_bush1
my_bush2
אחירותם
חרוב מצוי
צבעוני ההרים


show anames
from the plantingmap
group by anames
where aname not exiest in
    welect pnames from plantingmap as p2
    where p2.aname = 'שדרת הזמיר'
    and p2.aname <> p1.aname


לא קיים בהן צמח שלא קיים בשדרות הזמיר


    The GROUP BY should come after the WHERE clause.
    The exists operator is used to determine the existence of rows in a subquery. It doesn't work directly with column values like in. The subquery with exists should be constructed using a correlated subquery.
    In your subquery, you have written p1.panme which seems to be a typo. It should probably be p1.pname.
    It's generally better to structure your query in a more readable format.

Let me correct these issues and provide you with a new query:

```sql

SELECT DISTINCT p1.aname
FROM plantingmap AS p1
WHERE NOT EXISTS (
    SELECT 1
    FROM plantingmap AS p2
    WHERE p2.aname = 'שדרות הזמיר'
      AND p2.pname NOT IN (
          SELECT pname
          FROM plantingmap
          WHERE aname = p1.aname
      )
);
```
This query selects all aname values where there isn't a plant
 (pname) in 'שדרות הזמיר' that doesn't exist in that aname's list of plants.


In [14]:
%%sql
--3.25 find anames that have at least all plants from 'שדרות הזמיר'
-- wrong!!! I write "exist" instead of "exists"
select distinct p1.aname
from plantingmap as p1
group by aname
where p1.pname not exists in
    (select p1.panme from plantingmap as p2
    where p2.aname = 'שדרות הזמיר'
    and p2.pname <> p1.pname);


 * sqlite:///gardening.db
(sqlite3.OperationalError) near "where": syntax error
[SQL: --3.25 find anames that have at least all plants from 'שדרות הזמיר'
-- wrong!!! I write "exist" instead of "exists"
select distinct p1.aname
from plantingmap as p1
group by aname
where p1.pname not exists in
    (select p1.panme from plantingmap as p2
    where p2.aname = 'שדרות הזמיר'
    and p2.pname <> p1.pname);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [15]:
%%sql

SELECT DISTINCT p1.aname
FROM plantingmap AS p1
WHERE NOT EXISTS (
    SELECT 1
    FROM plantingmap AS p2
    WHERE p2.aname = 'שדרות הזמיר'
    -- wrong !!! and p2.pname <> p1.pname
    AND p2.pname NOT IN (
        SELECT pname
        FROM plantingmap
        where aname = p1.aname
    )
)

 * sqlite:///gardening.db
Done.


aname
הטיילת
שדרות הזמיר


In [16]:
%%sql
-- 3.25
SELECT DISTINCT P.aname
FROM plantingmap AS P
WHERE NOT EXISTS (SELECT 0
                  FROM plantingmap
                  WHERE aname = 'שדרות הזמיר'
                    AND pname NOT IN 
                     (SELECT P1.pname
                      FROM plantingmap as P1
                      where P.aname = P1.aname)
                 );

 * sqlite:///gardening.db
Done.


aname
הטיילת
שדרות הזמיר


-- 3.26: 3.24 - find plant that exist in at most 1 anme
-- now with unique

wrong! its possible that a plant wont be in any aname!!!!

In [17]:
%%sql

select aname
from plantingmap
where unique (select p1.pname
                from plantingmap as p1
                where p1.aname <> aname
                  and p1.pname = pname);

 * sqlite:///gardening.db
(sqlite3.OperationalError) near "unique": syntax error
[SQL: select aname
from plantingmap
where unique (select p1.pname
                from plantingmap as p1
                where p1.aname <> aname
                  and p1.pname = pname);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [18]:
%%sql

select pname
from plant as P1
where unique (select pname
              from palntingmap as P2
              where P1.pname = P2.pname  );

 * sqlite:///gardening.db
(sqlite3.OperationalError) near "unique": syntax error
[SQL: select pname
from plant as P1
where unique (select pname
              from palntingmap as P2
              where P1.pname = P2.pname  );]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


# EDITING DB

In [20]:
%%sql
INSERT INTO plant (pname, ptype, pdiameter, pmaxheight) 
VALUES ('Sunflower', 'flower', 10, 20);

 * sqlite:///gardening.db
1 rows affected.


[]

In [21]:
%%sql
delete from plant
where pname = 'Sunflower';

 * sqlite:///gardening.db
1 rows affected.


[]

In [22]:
%%sql

insert into plant
    values ('שזיף דובדבני', 'עץ נוי', 4.00, 7.00),
            ('כליל החורש', 'עץ נוי', 3.00, 4.00);

 * sqlite:///gardening.db
2 rows affected.


[]

In [24]:
%%sql

insert into plantingmap
    select 'שזיף דובדבני', aname, amount
    from plantingmap
    where pname = 'אורן ירושלים';

 * sqlite:///gardening.db
4 rows affected.


[]

In [25]:
%%sql

delete from plantingmap
where pname = 'אורן ירושלים';

 * sqlite:///gardening.db
4 rows affected.


[]

In [26]:
%%sql

update plantingmap
set pname = 'אורן ירושלים'
where pname =  'שזיף דובדבני';

 * sqlite:///gardening.db
4 rows affected.


[]

# join

In [27]:
%%sql

select *
from plantingmap, area
where plantingmap.aname = area.aname;

 * sqlite:///gardening.db
Done.


pname,aname,amount,aname_1,alocation,atype,asize
זית,גן העצמאות,10,גן העצמאות,רחוב העצמאות,פארק נופש פעיל,60000
ברוש,גן העצמאות,30,גן העצמאות,רחוב העצמאות,פארק נופש פעיל,60000
חבושית,גן העצמאות,200,גן העצמאות,רחוב העצמאות,פארק נופש פעיל,60000
לנטנה לילכית,גן העצמאות,150,גן העצמאות,רחוב העצמאות,פארק נופש פעיל,60000
ורד,גן העצמאות,130,גן העצמאות,רחוב העצמאות,פארק נופש פעיל,60000
ציפורני חתול,גן העצמאות,250,גן העצמאות,רחוב העצמאות,פארק נופש פעיל,60000
חרוב מצוי,כיכר הכניסה,5,כיכר הכניסה,כיכר הכניסה,פארק נופש פעיל,2500
זית,כיכר הכניסה,7,כיכר הכניסה,כיכר הכניסה,פארק נופש פעיל,2500
רוזמרין,כיכר הכניסה,50,כיכר הכניסה,כיכר הכניסה,פארק נופש פעיל,2500
ורד,כיכר הכניסה,50,כיכר הכניסה,כיכר הכניסה,פארק נופש פעיל,2500


In [28]:
%%sql

select *
from plantingmap natural join area;

 * sqlite:///gardening.db
Done.


pname,aname,amount,alocation,atype,asize
זית,גן העצמאות,10,רחוב העצמאות,פארק נופש פעיל,60000
ברוש,גן העצמאות,30,רחוב העצמאות,פארק נופש פעיל,60000
חבושית,גן העצמאות,200,רחוב העצמאות,פארק נופש פעיל,60000
לנטנה לילכית,גן העצמאות,150,רחוב העצמאות,פארק נופש פעיל,60000
ורד,גן העצמאות,130,רחוב העצמאות,פארק נופש פעיל,60000
ציפורני חתול,גן העצמאות,250,רחוב העצמאות,פארק נופש פעיל,60000
חרוב מצוי,כיכר הכניסה,5,כיכר הכניסה,פארק נופש פעיל,2500
זית,כיכר הכניסה,7,כיכר הכניסה,פארק נופש פעיל,2500
רוזמרין,כיכר הכניסה,50,כיכר הכניסה,פארק נופש פעיל,2500
ורד,כיכר הכניסה,50,כיכר הכניסה,פארק נופש פעיל,2500


WRONG - didnt specipid the aname after "join on..."
```sql
%%sql

select alocation as CHARUV_planting_spots
from plantingmap join area on aname           <=======
where pname = 'חרוב מצוי';

In [31]:
%%sql

select alocation as CHARUV_planting_spots
from plantingmap join area on plantingmap.aname = area.aname
where pname = 'חרוב מצוי';

 * sqlite:///gardening.db
Done.


CHARUV_planting_spots
כיכר הכניסה


In [37]:
%%sql

select plantingmap.aname, asize, sum(amount)
from plantingmap join area on plantingmap.aname = area.aname
group by plantingmap.aname;

 * sqlite:///gardening.db
Done.


aname,asize,sum(amount)
בריכת המים,400,110
גינת הגפן,3600,55
גן העצמאות,60000,790
הטיילת,10000,230
"חצר המתנ""ס",1500,55
כיכר הכניסה,2500,112
כיכר העיריה,5000,61
כיכר הרכבת,3000,54
שדרות הזמיר,900,90
שדרת הראשונים,1500,400
